### Leer todos los datos que son requeridps

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_foler_path}/movies") \
                        .filter(f"file_date = '{v_file_date}'")

In [0]:
country_df = spark.read.format("delta").load(f"{silver_foler_path}/country")

In [0]:
production_country_df = spark.read.format("delta").load(f"{silver_foler_path}/production_country") \
                        .filter(f"file_date = '{v_file_date}'")

#### Join "country" y "production_country"

In [0]:
country_prod_coun_df = country_df.join(production_country_df,
                                       country_df.country_id == production_country_df.country_id,
                                       "inner") \
                                    .select(country_df.country_name,
                                            production_country_df.movie_id)

#### Join "movies_df" y "country_prod_coun_df"

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2015")

In [0]:
results_movies_country_prod_coun_df = movies_filter_df.join(country_prod_coun_df,
                                            movies_filter_df.movie_id == country_prod_coun_df.movie_id,
                                            "inner")

In [0]:
results_df = results_movies_country_prod_coun_df.select("year_release_date","country_name",
                                                        "budget","revenue")

In [0]:
from pyspark.sql.functions import sum

results_group_by_df = results_df \
                    .groupBy("year_release_date","country_name") \
                    .agg(
                        sum("budget").alias("total_budget"),
                        sum("revenue").alias("total_revenue")
                    )

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, lit

results_dense_rank_df = Window.partitionBy("year_release_date").orderBy(
                                                                desc("total_budget"),
                                                                desc("total_revenue"))
final_df = results_group_by_df.withColumn("dense_rank",dense_rank().over(results_dense_rank_df)) \
                              .withColumn("created_date",lit(v_file_date))

#### Escribir datos en el Datalake en formato "Parquet"

In [0]:
merge_condition = 'tgt.year_release_date = src.year_release_date AND tgt.country_name = src.country_name AND tgt.created_date = src.created_date'
merge_delta_lake(final_df, "movie_gold", "results_group_country", gold_folder_path, merge_condition, "created_date")

#overwrite_partition(final_df,"movie_gold","results_group_country","created_date")

In [0]:
%sql
SELECT * 
FROM movie_gold.results_group_country